In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10000)


import altair as alt
alt.data_transformers.disable_max_rows()

import glob
import datetime
import seaborn as sns
import matplotlib.pyplot as plt

import act
import pyart
import xarray as xr
import os



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [2]:
username = os.getenv("ARM_USERNAME")
token = os.getenv("ARM_TOKEN")

# Identify datastreams

In [5]:
# ppi:  Single-pass full-360o plan position indicator scan 
# ppi2: Single- or multi-pass full or limited sector plan position indicator scan 
# rhi:  Single-pass full-180o range height indicator scan 
# rhi2: Single- or multi-pass full or limited sector range height indicator scan
ds_dl_ppi = 'gucdlppiM1.b1'
ds_dl_rhi = 'gucdlrhiM1.b1'
ds_dl_rhi2 = 'gucdlrhi2M1.b1'

startdate = '2022-10-30'
enddate = '2022-11-04'

# Download datasets

In [ ]:
act.discovery.download_data(username, token, 'dlprofwind4news', startdate, enddate)

In [ ]:
act.discovery.download_data(username, token, ds_dl_ppi, startdate, enddate)
act.discovery.download_data(username, token, ds_dl_rhi, startdate, enddate)
act.discovery.download_data(username, token, ds_dl_rhi2, startdate, enddate)

In [ ]:
startdate1 = '2022-10-30'
enddate1 = '2022-11-01'
startdate2 = '2022-11-02'
enddate2 = '2022-11-04'
act.discovery.download_data(username, token, ds_dl_ppi, startdate1, enddate1)
act.discovery.download_data(username, token, ds_dl_rhi, startdate1, enddate1)
act.discovery.download_data(username, token, ds_dl_rhi2, startdate1, enddate1)

In [ ]:
act.discovery.download_data(username, token, ds_dl_ppi, startdate2, enddate2)
act.discovery.download_data(username, token, ds_dl_rhi, startdate2, enddate2)
act.discovery.download_data(username, token, ds_dl_rhi2, startdate2, enddate2)

In [6]:
# dl_ppi_files = glob.glob('gudlppiM1.b1')
# dl_rhi_files = glob.glob('gucdlrhiM1.b1')

dl_ppi_files = glob.glob(''.join(['./',ds_dl_ppi,'/*cdf']))
dl_rhi_files = glob.glob(''.join(['./',ds_dl_rhi,'/*cdf']))
dl_rhi2_files = glob.glob(''.join(['./',ds_dl_rhi2,'/*cdf']))

In [7]:
dl_rhi = act.io.armfiles.read_netcdf(dl_rhi_files)

In [8]:
dl_ppi = act.io.armfiles.read_netcdf(dl_ppi_files)


# Examine RHI data

In [9]:
src_rhi = dl_rhi.to_dataframe().reset_index()

# Convert time zone

In [10]:
src_rhi['time'] = src_rhi['time'] - datetime.timedelta(hours = 6)

In [11]:
(
    len(src_rhi['time'].unique()), 
    len(src_rhi['range'].unique()), 
    len(src_rhi['elevation'].unique()), 
    len(src_rhi['azimuth'].unique())
)

(43561, 400, 14224, 2)

In [12]:
(
    # len(src_rhi['elevation'].unique()), 
    len(src_rhi['azimuth'].unique())
)

2

43561 timestamps over multiple days  (based on our query)

400 discrete range values

13866 discrete elevation angles

2 azimuth directions (0, 270)

In [13]:
(
    src_rhi['time'].unique(), 
    src_rhi['range'].unique(), 
    src_rhi['elevation'].unique(), 
    src_rhi['azimuth'].unique()
)

(array(['2022-10-29T18:01:18.699000000', '2022-10-29T18:01:19.869000000',
        '2022-10-29T18:01:20.889000000', ...,
        '2022-11-03T17:07:35.781000000', '2022-11-03T17:07:36.790000000',
        '2022-11-03T17:07:37.930000000'], dtype='datetime64[ns]'),
 array([   15.,    45.,    75.,   105.,   135.,   165.,   195.,   225.,
          255.,   285.,   315.,   345.,   375.,   405.,   435.,   465.,
          495.,   525.,   555.,   585.,   615.,   645.,   675.,   705.,
          735.,   765.,   795.,   825.,   855.,   885.,   915.,   945.,
          975.,  1005.,  1035.,  1065.,  1095.,  1125.,  1155.,  1185.,
         1215.,  1245.,  1275.,  1305.,  1335.,  1365.,  1395.,  1425.,
         1455.,  1485.,  1515.,  1545.,  1575.,  1605.,  1635.,  1665.,
         1695.,  1725.,  1755.,  1785.,  1815.,  1845.,  1875.,  1905.,
         1935.,  1965.,  1995.,  2025.,  2055.,  2085.,  2115.,  2145.,
         2175.,  2205.,  2235.,  2265.,  2295.,  2325.,  2355.,  2385.,
         2415.,  24

## RHI: convert polar coordinates to rectangular coords with the radar at (0,0)

In [14]:
src_rhi['x'] = src_rhi['range']*np.cos(np.deg2rad(src_rhi['elevation']))
src_rhi['z'] = src_rhi['range']*np.sin(np.deg2rad(src_rhi['elevation']))

## Plot hourly RHI scans (azimuth == 0)

In [15]:
src_rhi['date'] = src_rhi['time'].dt.date

In [16]:
src_rhi['date'].min(), src_rhi['date'].max()

(datetime.date(2022, 10, 29), datetime.date(2022, 11, 3))

In [18]:
src_rhi.query('range==15').query('qc_time==0').head(400)

,time,range,base_time,time_offset,qc_time,azimuth,elevation,radial_velocity,qc_radial_velocity,intensity,attenuated_backscatter,lat,lon,alt,x,z,date
400,2022-10-29 18:01:19.869,15.0,2022-10-30,2022-10-30 00:01:19.869,0,0.0,0.010000,-0.6497,0,0.902966,-5.464560e-06,38.956158,-106.987854,2886.0,15.000000,0.002618,2022-10-29
800,2022-10-29 18:01:20.889,15.0,2022-10-30,2022-10-30 00:01:20.889,0,0.0,1.060000,-0.7262,0,0.983314,-9.396753e-07,38.956158,-106.987854,2886.0,14.997433,0.277491,2022-10-29
1200,2022-10-29 18:01:21.910,15.0,2022-10-30,2022-10-30 00:01:21.910,0,0.0,2.070000,-0.5733,0,0.825463,-9.829202e-06,38.956158,-106.987854,2886.0,14.990212,0.541807,2022-10-29
1600,2022-10-29 18:01:22.959,15.0,2022-10-30,2022-10-30 00:01:22.959,0,0.0,3.120000,-0.6497,0,0.982479,-9.867298e-07,38.956158,-106.987854,2886.0,14.977766,0.816410,2022-10-29
2000,2022-10-29 18:01:23.980,15.0,2022-10-30,2022-10-30 00:01:23.980,0,0.0,4.140000,-0.5733,0,0.943326,-3.191624e-06,38.956158,-106.987854,2886.0,14.960859,1.082906,2022-10-29
2400,2022-10-29 18:01:25.039,15.0,2022-10-30,2022-10-30 00:01:25.039,0,0.0,5.200000,-0.7262,0,0.964312,-2.009800e-06,38.956158,-106.987854,2886.0,14.938266,1.359489,2022-10-29
2800,2022-10-29 18:01:26.060,15.0,2022-10-30,2022-10-30 00:01:26.060,0,0.0,6.200000,-0.6880,0,0.998431,-8.834754e-08,38.956158,-106.987854,2886.0,14.912264,1.619990,2022-10-29
3200,2022-10-29 18:01:27.080,15.0,2022-10-30,2022-10-30 00:01:27.080,0,0.0,7.230000,-0.5351,0,0.765451,-1.320884e-05,38.956158,-106.987854,2886.0,14.880734,1.887790,2022-10-29
3600,2022-10-29 18:01:28.099,15.0,2022-10-30,2022-10-30 00:01:28.099,0,0.0,8.230000,-0.7262,0,1.002448,1.378785e-07,38.956158,-106.987854,2886.0,14.845521,2.147207,2022-10-29
4000,2022-10-29 18:01:29.129,15.0,2022-10-30,2022-10-30 00:01:29.129,0,0.0,9.260000,-0.6497,0,1.069670,3.923531e-06,38.956158,-106.987854,2886.0,14.804524,2.413722,2022-10-29


In [ ]:
src = src_rhi[src_rhi['azimuth'] == 0].query('range < 1000')
plot_hours = [
    0,2,4,6,8,  10, 11, 12, 13, 14, 16, 18, 20, 22
]
plot_days = list(sorted(src_rhi['date'].unique()))

fig, axes = plt.subplots(
    len(src_rhi['date'].unique()), 
    len(plot_hours), 
    figsize=(30,10), 
    sharex=True, sharey=True
)


for i_day, day in enumerate(plot_days):
    for i,hr in enumerate(plot_hours):
        local_src = src[ (src['date'] == day) & (src['time'].dt.hour == hr)]
        ax = axes[i_day][i]
        hexplot = ax.hexbin(local_src['x'], local_src['z'], C=local_src['radial_velocity'], cmap='RdYlBu', clim=(-10, 10))
        # ax.annotate(f"{day}:00", xy=(-900, 900))
        ax.title.set_text(f"{day} {hr}:00")
        ax.title.set_fontsize(8)
        ax.set_xlim(-1000, 1000)
        ax.set_ylim(0, 1000)
fig.colorbar(hexplot, ax=axes.ravel().tolist())

In [ ]:
src = src_rhi[src_rhi['azimuth'] == 0].query('range < 1000')
plot_hours = [
    11,12,13,16,17,18,19,21,22,23
]
plot_days = list(sorted(src_rhi['date'].unique()))

fig, axes = plt.subplots(
    len(src_rhi['date'].unique()), 
    len(plot_hours), 
    figsize=(30,10), 
    sharex=True, sharey=True
)


for i_day, day in enumerate(plot_days):
    for i,hr in enumerate(plot_hours):
        local_src = src[ (src['date'] == day) & (src['time'].dt.hour == hr)]
        ax = axes[i_day][i]
        hexplot = ax.hexbin(local_src['x'], local_src['z'], C=local_src['radial_velocity'], cmap='RdYlBu', clim=(-10, 10))
        # ax.annotate(f"{day}:00", xy=(-900, 900))
        ax.title.set_text(f"{day} {hr}:00")
        ax.title.set_fontsize(8)
        ax.set_xlim(-1000, 1000)
        ax.set_ylim(0, 1000)
fig.colorbar(hexplot, ax=axes.ravel().tolist())

In [ ]:
import matplotlib
font = {
    # 'family' : 'normal',
    #     'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

In [ ]:
src = src_rhi[src_rhi['azimuth'] == 0].query('range < 1000')
src = src[(src['date'] == datetime.date(2022, 11, 1)) & (src['time'].dt.hour.isin([18]))]

hexplot = plt.hexbin(src['x'], src['z'], C=src['radial_velocity'], cmap='RdYlBu', clim=(-10, 10))
plt.xlim(-1000, 1000)
plt.ylim(0, 1000)
cbar = plt.colorbar()
cbar.set_label('Radial velocity (m/s)')
plt.xticks([-1000,0,1000])
plt.tick_params(direction='out', pad=10)

plt.yticks([0,500,1000])
plt.xlabel("Distance north from LIDAR (m)")
plt.ylabel("Altitude (m)")
assert len(src['date'].unique()) == 1

# plt.title(
#     f"{src['time'].min()} to {src['time'].max()}",
#     y=1.05
# )

## Plot hourly RHI scans (azimuth == 270)

In [ ]:
src = src_rhi[src_rhi['azimuth'] == 270].query('range < 1000')
plot_hours = [
    11,12,13,16,17,18,19,21,22,23
]
plot_days = list(sorted(src_rhi['date'].unique()))

fig, axes = plt.subplots(
    len(src_rhi['date'].unique()), 
    len(plot_hours), 
    figsize=(30,10), 
    sharex=True, sharey=True
)


for i_day, day in enumerate(plot_days):
    for i,hr in enumerate(plot_hours):
        local_src = src[ (src['date'] == day) & (src['time'].dt.hour == hr)]
        ax = axes[i_day][i]
        hexplot = ax.hexbin(local_src['x'], local_src['z'], C=local_src['radial_velocity'], cmap='RdYlBu')
        # ax.annotate(f"{day}:00", xy=(-900, 900))
        ax.title.set_text(f"{day} {hr}:00")
        ax.title.set_fontsize(8)
        ax.set_xlim(-1000, 1000)
        ax.set_ylim(0, 1000)
fig.colorbar(hexplot, ax=axes.ravel().tolist())

# Examine PPI data

## By using xarray DIRECTLY

In [ ]:
dl_ppi_files[:5], dl_ppi_files[-5:]

In [ ]:
df = xr.open_dataset(dl_ppi_files[0]).to_dataframe().reset_index()

In [ ]:
dl_ppi.time.max(), dl_ppi.time.min()

## By using the ACT library to compute vertical wind profiles

In [ ]:
len(dl_ppi[dict(range=slice(0, 2))].to_dataframe()), len(dl_ppi.to_dataframe())

### Limit range values so that computing winds doesn't take so long

In [ ]:
dl_ppi_slice = dl_ppi[dict(range=slice(0, 4))].sel(
    time=slice(datetime.datetime(2022,11,2,4), datetime.datetime(2022,11,2,16))
)

In [ ]:
# Calculate the winds from the gucdlppi dataset.
wind_obj_sliced = act.retrievals.compute_winds_from_ppi(
    dl_ppi_slice,
    remove_all_missing=True, 
    snr_threshold=0.002
)

In [ ]:
wind_obj_sliced.to_dataframe()

In [ ]:
alt.Chart(
    wind_obj_sliced.to_dataframe().reset_index()
).mark_point(shape="wedge", filled=True).encode(
    alt.X('time:T'),
    alt.Y('height:Q'),
    # color=alt.Color(
    #     "wind direction", scale=alt.Scale(domain=[0, 360], scheme="rainbow"), legend=None
    # ),
    angle=alt.Angle("wind_direction", scale=alt.Scale(domain=[0, 360], range=[180, 540])),
    size=alt.Size("wind_speed", scale=alt.Scale(rangeMax=2000, rangeMin=250), title='Wind Speed (m/s)'),
).properties(
    width=1000,
    height=200
)

In [ ]:
(
    alt.Chart(
        wind_obj_sliced.to_dataframe().reset_index()
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y('wind_direction:Q', title='Wind Direction'),
        alt.Color('height:N')
    ).properties(width=1000, height=200)
&
    alt.Chart(
        wind_obj_sliced.to_dataframe().reset_index()
    ).transform_window(
        rolling_mean='mean(wind_direction)',
        frame=[-4, 4],
        groupby=['height']
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y('rolling_mean:Q'),
        alt.Color('height:N')
    ).properties(width=1000, height=200)
)

### For all range values

In [ ]:
# Calculate the winds from the gucdlppi dataset.
wind_obj = act.retrievals.compute_winds_from_ppi(
    dl_ppi, remove_all_missing=True, snr_threshold=0.002)

In [ ]:
ppi_calc_winds_df = wind_obj.to_dataframe()

In [ ]:
src = ppi_calc_winds_df.query("height < 100")

In [ ]:
alt.Chart(src.reset_index()).transform_window(
    rolling_mean='mean(wind_direction)',
    frame=[-4, 4],
    groupby=['height']
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('rolling_mean:Q'),
    alt.Color('height:N')
).properties(width=1000)


In [ ]:
alt.Chart(src.reset_index()).mark_line().encode(
    alt.X('time:T'),
    alt.Y('wind_direction'),
    alt.Color('height:N')
).properties(width=1400)

In [ ]:
# Create a display object.
display = act.plotting.TimeSeriesDisplay(
    {"GUC DLPPI Computed Winds over KAZR": wind_obj,
     }, figsize=(20, 10))

display.plot_barbs_from_spd_dir('wind_speed', 'wind_direction',
                                dsname='GUC DLPPI Computed Winds over KAZR',
                                invert_y_axis=False)

# Update the x-limits to make sure both wind profiles are shown
display.axes[0].set_xlim([np.datetime64('2022-08-01T22:10'), np.datetime64('2022-08-01T23:50')])
plt.show()

In [ ]:
(
    len(ppi_calc_winds_df['time'].unique()), 
    len(ppi_calc_winds_df['height'].unique()), 
),(
    ppi_calc_winds_df['time'].unique(), 
    ppi_calc_winds_df['height'].unique(), 
)

8 timestamps over ~36 seconds, every ~5 seconds

400 discrete range values

1 elevation angle

8 azimuth directions (259.1, 304.1, 349.1,  34.100006,  79.100006, 124.100006, 169.09998 , 214.09998)

In [ ]:
(
    len(df['time'].unique()), 
    len(df['range'].unique()), 
    len(df['elevation'].unique()), 
    len(df['azimuth'].unique())
),(
    df['time'].unique(), 
    df['range'].unique(), 
    df['elevation'].unique(), 
    df['azimuth'].unique()
)

In [ ]:
df['azimuth'].unique()

# Look at XPrecip radar data, PPI

In [ ]:
df2 = xr.open_dataset('/Users/elischwat/Downloads/gucxprecipradarS2.00.20221030.000005.raw.nc/gucxprecipradarS2.00.20221030.000130.raw.csu.sail-20221030-000130_702887_22_1_PPI.nc').to_dataframe()

from 

https://cookbooks.projectpythia.org/radar-cookbook/notebooks/foundations/pyart-gridding.html

In [ ]:
radar = pyart.io.read('/Users/elischwat/Downloads/gucxprecipradarS2.00.20221030.000005.raw.nc/gucxprecipradarS2.00.20221030.000130.raw.csu.sail-20221030-000130_702887_22_1_PPI.nc')
print(radar.scan_type)

In [ ]:
fig = plt.figure(figsize=[10, 10])
display = pyart.graph.RadarMapDisplay(radar)
radar.fields.keys()

In [ ]:
display.plot_ppi('DBZ')

# Look at XPrecip radar data, RHI

In [ ]:
# Explore files

radar = pyart.io.read('/Users/elischwat/Downloads/gucxprecipradarS2.00.20221030.000005.raw.nc/gucxprecipradarS2.00.20221030.000005.raw.csu.sail-20221030-000005_702875_22_326_RHI.nc')
print(radar.scan_type)

fig = plt.figure(figsize=[10, 10])
display = pyart.graph.RadarMapDisplay(radar)
radar.fields.keys()

In [ ]:
display.plot_rhi('DBZ')